# Streaming Agents with Callbacks (Advanced)

This notebooks shows how to stream agent using callbacks.

This notebook uses an agent based on OpenAI tools invocation.

In [4]:
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from uuid import UUID

from langchain import agents, hub
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.callbacks.base import AsyncCallbackHandler
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGenerationChunk, GenerationChunk, LLMResult
from langchain_openai import ChatOpenAI

In [5]:
model = ChatOpenAI(temperature=0)

In [14]:
@tool
def what_does_the_cat_say(callbacks) -> str:
    """What does the cat say?!"""
    chunks = list(
        model.stream(
            "You are a cat. Say something that a cat may say. make it cat like.",
            {"callbacks": callbacks, "tags": ["cat_says"]},
        )
    )
    return "".join(chunk.content for chunk in chunks)


@tool
def where_cat_is_hiding(callbacks) -> str:
    """Where is the cat hiding right now?"""
    chunks = list(
        model.stream(
            "Give one up to three word answer about where the cat might be hiding in the house right now.",
            {"callbacks": callbacks, "tags": ["hiding_spot"]},
        )
    )
    return "".join(chunk.content for chunk in chunks)


@tool
def tell_me_a_joke_about(topic: str, callbacks) -> str:
    """Tell a joke about a given topic."""
    template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are Cat Agent 007. You are funny and know many jokes."),
            ("human", "Tell me a joke about {topic}"),
        ]
    )
    chain = template | model.with_config({"tags": ["joke"]})
    chunks = list(chain.stream({"topic": topic}, {"callbacks": callbacks}))
    return "".join(chunk.content for chunk in chunks)

In [15]:
class CallbackHandler(AsyncCallbackHandler):
    def __init__(self, filter_tags: List[str]) -> None:
        """Initialize a callback handler."""
        self.filter_tags = filter_tags

    async def on_chat_model_start(
        self,
        serialized: Dict[str, Any],
        messages: List[List[BaseMessage]],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> Any:
        """Run when a chat model starts running."""
        if not self.has_overlap(tags):
            return
        print()
        print("-----")
        print(tags)

    def has_overlap(self, tags: Optional[List[str]]) -> bool:
        """Check for overlap with filtered tags."""
        if not tags:
            return False
        return bool(set(tags or []) & set(self.filter_tags or []))

    async def on_llm_new_token(
        self,
        token: str,
        *,
        chunk: Optional[Union[GenerationChunk, ChatGenerationChunk]] = None,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> None:
        """Run on new LLM token. Only available when streaming is enabled."""
        if not self.has_overlap(tags):
            return
        print(token, end="")

In [16]:
handler = CallbackHandler(filter_tags=["cat_says", "hiding_spot", "joke"])

In [17]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [18]:
tools = [what_does_the_cat_say, tell_me_a_joke_about, where_cat_is_hiding]
agent = agents.create_openai_tools_agent(
    model.with_config({"tags": ["agent"]}), tools, prompt
)
executor = agents.AgentExecutor(agent=agent, tools=tools)

In [23]:
executor.invoke({"input": "what does the cat say?"}, {"callbacks": [handler]})


-----
['cat_says']
Meow, hooman! I demand your immediate attention and affection.

In [24]:
executor.invoke(
    {"input": "tell me a joke about the location where the cat is hiding"},
    {"callbacks": [handler]},
)


-----
['hiding_spot']
Under the bed.
-----
['seq:step:2', 'joke']
Why did the scarecrow bring a ladder under the bed?

Because it heard there was a "bed spring" party going on!

In [25]:
handler = CallbackHandler(filter_tags=["cat_says", "hiding_spot", "joke"])
executor.invoke(
    {"input": "tell me a joke about the location where the cat is hiding"},
    {"callbacks": [handler]},
)


-----
['hiding_spot']
Under the bed.
-----
['seq:step:2', 'joke']
Why did the scarecrow bring a ladder under the bed?

Because it heard there was a "bed spring" party going on!